In [66]:
# refactor with pandas
import pandas as pd
import numpy as np
import math

In [2]:
# manually update to reflect most up-to-date week
matchups = pd.read_csv("2017_yahoo_matchups_week1-10.csv",delimiter='|')
weeks_elapsed = 10



In [3]:
matchups

,Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo,Year
0,1,The Golden Yogis,116.14,84.22,<,114.70,114.64,Sean McBae,2017
1,1,Clippernation,113.36,93.12,>,91.72,120.17,TeamLC,2017
2,1,Butkicker&Tygod,120.96,69.84,<,81.12,100.64,LordN’Savior Josh G,2017
3,1,Sit4Anthems,118.75,114.94,>,91.58,118.03,BigBradyBrand,2017
4,1,Team 8,122.70,121.84,>,98.98,109.05,Healthyplayersplz,2017
5,2,The Golden Yogis,120.19,109.52,<,110.62,120.81,Team 8,2017
6,2,Clippernation,112.58,125.82,>,100.42,119.05,Butkicker&Tygod,2017
7,2,Sean McBae,115.00,103.96,<,128.18,122.92,LordN’Savior Josh G,2017
8,2,TeamLC,125.30,90.26,>,65.40,109.84,Sit4Anthems,2017
9,2,BigBradyBrand,118.63,87.88,<,95.58,117.55,Healthyplayersplz,2017


In [4]:
# sort by score per week
matchups_stacked = pd.DataFrame()

for i in range(1, weeks_elapsed+1): 
    # alternatively matchups[["Week"], ...].filter_by(1, "Week")
    matchup_week_score = matchups[matchups["Week"]==i][["Week", "TeamOne", "ProjectedScoreOne", "ScoreOne" ]] 
    matchup_week_score.rename(columns = {"TeamOne":"Team", "ProjectedScoreOne":"ProjectedScore", "ScoreOne":"Score"}, 
                              inplace = True)
    
    matchup_week_score_two = matchups[matchups["Week"]==i][["Week", "TeamTwo", "ProjectedScoreTwo", "ScoreTwo" ]] 
    matchup_week_score_two.rename(columns = {"TeamTwo":"Team", "ProjectedScoreTwo":"ProjectedScore", "ScoreTwo":"Score"}, 
                                  inplace = True)

    matchup_week_score = matchup_week_score.append(matchup_week_score_two)
    matchup_week_score = matchup_week_score.sort_values(["Score"]) \
    .assign(WeeklyBeaten = range(matchup_week_score.shape[0]))
    
    matchups_stacked = matchups_stacked.append(matchup_week_score)

matchups_stacked = matchups_stacked.reset_index(drop=True)

matchups_stacked.head()

,Week,Team,ProjectedScore,Score,WeeklyBeaten
0,1,Butkicker&Tygod,120.96,69.84,0
1,1,LordN’Savior Josh G,100.64,81.12,1
2,1,The Golden Yogis,116.14,84.22,2
3,1,BigBradyBrand,118.03,91.58,3
4,1,TeamLC,120.17,91.72,4


## Expected Wins

In [119]:
# Expected Wins = TotalNumBeaten / Num_Opponents
# AvgBeatenPerWeek = TotalNumBeaten / Weeks

# next steps: include actual wins from API

num_opponents = matchups.shape[0] / weeks_elapsed * 2 - 1
 
expected_wins = matchups_stacked.groupby(["Team"], as_index = False).agg({"WeeklyBeaten": "sum"}) \
.rename(columns = {"WeeklyBeaten": "TotalBeaten"})

expected_wins = expected_wins.assign(ExpectedWins = expected_wins["TotalBeaten"] / num_opponents) \
.assign(AvgBeatPerWeek = expected_wins["TotalBeaten"] / weeks_elapsed) \
.sort_values("ExpectedWins", ascending = False)

expected_wins = expected_wins[["Team", "ExpectedWins", "AvgBeatPerWeek"]].round(2).reset_index(drop=True)

expected_wins



,Team,ExpectedWins,AvgBeatPerWeek
0,TeamLC,6.67,6.0
1,BigBradyBrand,6.00,5.4
2,Sean McBae,5.89,5.3
3,LordN’Savior Josh G,5.78,5.2
4,Clippernation,5.11,4.6
5,Team 8,4.89,4.4
6,Healthyplayersplz,4.56,4.1
7,Sit4Anthems,4.33,3.9
8,Butkicker&Tygod,3.67,3.3
9,The Golden Yogis,3.11,2.8


## Point Differentials

In [5]:
# Calculate Point Diff
# Point Diff is the score difference per matchup
matchups['Diff'] = abs(matchups['ScoreOne'] - matchups['ScoreTwo'])
matchups.head()

,Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo,Year,Diff
0,1,The Golden Yogis,116.14,84.22,<,114.70,114.64,Sean McBae,2017,30.48
1,1,Clippernation,113.36,93.12,>,91.72,120.17,TeamLC,2017,1.40
2,1,Butkicker&Tygod,120.96,69.84,<,81.12,100.64,LordN’Savior Josh G,2017,11.28
3,1,Sit4Anthems,118.75,114.94,>,91.58,118.03,BigBradyBrand,2017,23.36
4,1,Team 8,122.70,121.84,>,98.98,109.05,Healthyplayersplz,2017,22.86


### Biggest Blowouts

In [6]:
# Biggest Blowouts: top 10 point largest differentials in season
biggest_blowouts = matchups.nlargest(10, 'Diff')

# quick and dirty way to restack winners on one side: use lists
winner = list(biggest_blowouts["Winner"])
index = [x for x in range(0, len(winner)) if winner[x] == '<']
team_one = list(biggest_blowouts["TeamOne"])
team_two = list(biggest_blowouts["TeamTwo"])
score_one = list(biggest_blowouts["ScoreOne"])
score_two = list(biggest_blowouts["ScoreTwo"])

temp_team = list(team_one)
temp_score = list(score_one)

# swap team one and two and sign if winner is on right side, so winner is on left side
for i in index:
    team_one[i] = team_two[i]
    score_one[i] = score_two[i]
    team_two[i] = temp_team[i]
    score_two[i] = temp_score[i]
    winner[i] = '>'

biggest_blowouts["TeamOne"] = team_one
biggest_blowouts["TeamTwo"] = team_two
biggest_blowouts["ScoreOne"] = score_one
biggest_blowouts["ScoreTwo"] = score_two
biggest_blowouts["Winner"] = winner

biggest_blowouts

,Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo,Year,Diff
19,4,BigBradyBrand,122.59,151.28,>,83.70,129.99,LordN’Savior Josh G,2017,67.58
25,6,TeamLC,102.25,136.44,>,74.46,125.67,The Golden Yogis,2017,61.98
22,5,TeamLC,112.95,157.16,>,96.26,118.86,Sean McBae,2017,60.90
38,8,LordN’Savior Josh G,109.88,135.50,>,78.26,107.83,TeamLC,2017,57.24
10,3,BigBradyBrand,117.36,130.32,>,76.66,117.79,The Golden Yogis,2017,53.66
18,4,TeamLC,124.77,132.82,>,80.62,122.35,Healthyplayersplz,2017,52.20
17,4,Sean McBae,121.31,114.68,>,62.86,112.87,Butkicker&Tygod,2017,51.82
40,9,Butkicker&Tygod,97.26,118.60,>,68.32,102.36,The Golden Yogis,2017,50.28
39,8,BigBradyBrand,122.21,135.62,>,89.28,101.15,Team 8,2017,46.34
12,3,TeamLC,114.74,113.80,>,73.18,132.61,Butkicker&Tygod,2017,40.62


### Smallest Squeakers

In [7]:
# Smallest Squeakers: top 10 smallest point differentials in season
smallest_squeakers = matchups.nsmallest(10, 'Diff')

# quick and dirty way to restack winners on one side: use lists
winner = list(smallest_squeakers["Winner"])
index = [x for x in range(0, len(winner)) if winner[x] == '<']
team_one = list(smallest_squeakers["TeamOne"])
team_two = list(smallest_squeakers["TeamTwo"])
score_one = list(smallest_squeakers["ScoreOne"])
score_two = list(smallest_squeakers["ScoreTwo"])

temp_team = list(team_one)
temp_score = list(score_one)

# swap team one and two and sign if winner is on right side, so winner is on left side
for i in index:
    team_one[i] = team_two[i]
    score_one[i] = score_two[i]
    team_two[i] = temp_team[i]
    score_two[i] = temp_score[i]
    winner[i] = '>'

smallest_squeakers["TeamOne"] = team_one
smallest_squeakers["TeamTwo"] = team_two
smallest_squeakers["ScoreOne"] = score_one
smallest_squeakers["ScoreTwo"] = score_two
smallest_squeakers["Winner"] = winner

smallest_squeakers

,Week,TeamOne,ProjectedScoreOne,ScoreOne,Winner,ScoreTwo,ProjectedScoreTwo,TeamTwo,Year,Diff
5,2,Team 8,120.19,110.62,>,109.52,120.81,The Golden Yogis,2017,1.10
1,1,Clippernation,113.36,93.12,>,91.72,120.17,TeamLC,2017,1.40
30,7,LordN’Savior Josh G,116.21,93.62,>,90.74,110.73,The Golden Yogis,2017,2.88
33,7,Butkicker&Tygod,108.48,106.52,>,103.08,99.62,Sit4Anthems,2017,3.44
29,6,LordN’Savior Josh G,115.44,151.26,>,146.98,114.73,Healthyplayersplz,2017,4.28
45,10,The Golden Yogis,98.73,91.88,>,87.44,96.16,Clippernation,2017,4.44
35,8,Clippernation,122.94,93.52,>,87.96,99.51,The Golden Yogis,2017,5.56
49,10,LordN’Savior Josh G,102.75,97.58,>,90.60,101.97,Sit4Anthems,2017,6.98
9,2,Healthyplayersplz,118.63,95.58,>,87.88,117.55,BigBradyBrand,2017,7.70
14,3,Team 8,120.15,128.32,>,118.88,121.17,LordN’Savior Josh G,2017,9.44


## Avg Win/Loss Differential

In [107]:
# matchups.groupby(["TeamOne", "Winner"], ).agg({'Winner': "count"}).rename(columns = {"Winner": "Wins"}).reset_index()
matchups_flipped = matchups.assign(Winner_Flipped = ['>' if i == '<' else '<' for i in list(matchups['Winner'])])
m1 = matchups.groupby(["TeamOne", "Winner"], ).Diff.agg(['count', 'sum'])
m2 = matchups_flipped.groupby(["TeamTwo", "Winner_Flipped"], ).Diff.agg(['count', 'sum'])

midx1 = pd.MultiIndex.from_product([m1.index.levels[0], ['>', '<']])
midx2 = pd.MultiIndex.from_product([m2.index.levels[0], ['>', '<']])

matchup_wins_one = m1.reindex(midx1, fill_value = 0).reset_index() \
.rename(columns = {"level_0": "Team", "level_1": "W/L", "count": "Count", "sum": "Diff"})
matchup_wins_two = m2.reindex(midx2, fill_value = 0).reset_index() \
.rename(columns = {"level_0": "Team", "level_1": "W/L", "count": "Count", "sum": "Diff"})

#matchup_wins_two


In [108]:
# outer join in case a team is not in either side of the matchups

matchup_wins_stacked = matchup_wins_one.merge(matchup_wins_two, on=["Team", "W/L"], how="outer")
matchup_wins_stacked['Count_x'] = matchup_wins_stacked['Count_x'].fillna(0)
matchup_wins_stacked['Count_y'] = matchup_wins_stacked['Count_y'].fillna(0)
matchup_wins_stacked['Diff_x'] = matchup_wins_stacked['Diff_x'].fillna(0)
matchup_wins_stacked['Diff_y'] = matchup_wins_stacked['Diff_y'].fillna(0)

matchup_wins_stacked

matchup_wins_final = matchup_wins_stacked[['Team', 'W/L']] \
.assign(Count = matchup_wins_stacked['Count_x'] + matchup_wins_stacked['Count_y']) \
.assign(Diff = matchup_wins_stacked['Diff_x'] + matchup_wins_stacked['Diff_y']) \
.assign(AvgDiff = matchup_wins_final['Diff'] / matchup_wins_final['Count']) \
.round(2)

matchup_wins_final['Count'] = matchup_wins_final['Count'].astype('int')
matchup_wins_final['W/L'] = ["Wins" if k == ">" else "Losses" for k in matchup_wins_final['W/L']]



### Avg Differential by Team

In [109]:
matchup_wins_final

,Team,W/L,Count,Diff,AvgDiff
0,BigBradyBrand,Wins,5,208.76,41.75
1,BigBradyBrand,Losses,5,101.24,20.25
2,Butkicker&Tygod,Wins,5,120.24,24.05
3,Butkicker&Tygod,Losses,5,139.02,27.80
4,Clippernation,Wins,7,121.22,17.32
5,Clippernation,Losses,3,34.06,11.35
6,LordN’Savior Josh G,Wins,8,149.72,18.72
7,LordN’Savior Josh G,Losses,2,77.02,38.51
8,Sean McBae,Wins,6,172.64,28.77
9,Sean McBae,Losses,4,120.96,30.24


### Avg Differential by Wins and Losses

In [112]:
matchup_wins_final.sort_values(["W/L", "AvgDiff"], ascending = False).reset_index(drop=True)

,Team,W/L,Count,Diff,AvgDiff
0,TeamLC,Wins,7,295.38,42.20
1,BigBradyBrand,Wins,5,208.76,41.75
2,Sean McBae,Wins,6,172.64,28.77
3,Butkicker&Tygod,Wins,5,120.24,24.05
4,LordN’Savior Josh G,Wins,8,149.72,18.72
5,Sit4Anthems,Wins,3,55.38,18.46
6,Clippernation,Wins,7,121.22,17.32
7,The Golden Yogis,Wins,2,29.70,14.85
8,Team 8,Wins,4,49.68,12.42
9,Healthyplayersplz,Wins,3,31.66,10.55
